# Overview
This Jupyter notebook helps you create a 3D model using the Photoscan photogrammetry software, leveraging the computing power of a high-performance compute cluster. It assumes that your photographs are stored in a folder on [Box](http://berkeley.box.com), and the results of each step are saved back to Box.

If you want to manually edit the results of any step in the process, you can sync or download the project file and photos from Box to any computer with Photoscan installed (version 1.5 or higher), make your edits, and move the changes back to Box before proceeding with the next computationally-intensive processing step.

This Jupyter notebook should be considered as a test to develop recommendations and findings and not necessarily specific solutions that should be implemented as is. Rather, users should modify this notebook for further testing and experimentation and as is suitable to their specific needs, workflows, and frameworks.


### Software versions
This notebook has been tested using:
* python 3.5 kernel
* boxsdk (2.0.0a2)
* Photoscan 1.3 or later (for example, Photoscan 1.4.3)

## Workflow at a glance
The photogrammetry workflow laid out here involves the following steps. Each listed step has its own section in the notebook. 
1. **Overview**: the overview that you're currently reading
2. **Source Files**: defines where various source files are located.
3. **Box Setup**: connects this notebook to Box so it can access the photos you've stored there. 
4. **Project Setup**: pulls your images into the computing cluster, and creates a new Photoscan (psx) file if needed.
5. **Align Photos**: arranges the photos you've taken, relative to one another. 1% to 8% of the expected processing time.
6. **Build Dense Cloud**: creates the a point cloud from the photographs. The most computationally-intensive step, which can take 50%-96% of the total model generation time, depending on your settings. This step is run using GPU nodes on the cluster, which improve performance.
7. **Build Mesh**: connecting the point cloud together to build a mesh takes around 30% of the processing time on lower quality settings, or as little as 2% on higher quality settings.
8. **Build Texture**: building the texture takes between 8% (medium quality) and 1% (ultra high quality) of the processing time.
9. **Export Results**: exporting the photomosaic and OBJ files, and moving them to Box.

# Project Setup
This step connects the notebook to Box. Before you get started, make sure you have a folder in Box for your project. That folder should have a folder inside it called "images" that has all the photos you want to use for your 3D model. This workflow will also create a new folder, "files", that contains the project files-- you don't have to set that up in advance.

**MYashar: As far as things stand now, none of the code in this Jupyter notebook actually tries to access or download files from an "images" subfolder on Box or creates a new "files" folder containing the project files on Box. Rather,
image files are retrieved from the (top) Box project folder. In my case, I created a folder in Box called'myBoxFolderName' and put all of the *.jpg image files there, and this is where the code accesses and retrieves the image files on Box.**

Click the code box below, and then click the "run cell" button in the toolbar. This will run the code and move to the next box.

After you run the next code box, your user name on the cluster should appear at the bottom.

In [1]:
# !conda create -n py3.5 python=3.5
# !conda install -n py3.5 pip
!module load python/3.5
# !conda list
!source activate py3.5
! module load qt
# !pip install --user boxsdk==2.0.0a2
!python --version
!which python
!which python3
!python3 --version
!which jupyter
!jupyter nbextension enable --py widgetsnbextension
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
# !pip install boxsdk==2.0.0a2
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, VBox, interact_manual
import ipywidgets as widgets
from IPython.display import display
usernamearray = %sx whoami
username = usernamearray[0]
print('user name: ', username)

def f(x):
    return x

def pn(projectName):
    return projectName

def smf(mountFolder):
    return mountFolder

def bpf(boxFolder):
    return boxFolder

def scp(containerPath):
    return containerPath

def idd(imageDataPath):
    return imageDataPath

def hfp(homePath):
    return homePath

def rf(runPath):
    return runPath


#Define widgets and defaults
pntextbox=widgets.Text(value='projectname', description='Project Name')
# singularitymountfoldertextbox =widgets.Text(value='/scratch/', description='Singularity Mount Folder') 
singularitymountfoldertextbox =widgets.Text(value='/scratch/')
# singularitymountfoldertextbox =widgets.Text(value='/global/scratch/myashar/projectname/', description='Singularity Mount Folder') 
boxfoldertextbox =widgets.Text(value='myBoxFolderName', description='Box Project Folder') 
# Need to make sure that a folder called 'myBoxFolderName' is created in Box and the images are in that folder.
# scptextbox=widgets.Text(value='/global/scratch/groups/dh/photoscan13.img', description='Path to Container')
# scptextbox=widgets.Text(value='/global/scratch/groups/dh/photoscan_1_4_3.simg', description='Path to Container')
scptextbox=widgets.Text(value='/global/scratch/myashar/container/photoscan_1_4_3.simg', description='Path to Container')
iddtextbox=widgets.Text(value='/global/scratch/' + username + '/imageData', description='Image Data')
hometextbox=widgets.Text(value='/global/home/users/' + username + '/', description='Home Folder')
runtextbox=widgets.Text(value='/global/scratch/' + username + '/'  + pntextbox.value + '/', description='Run Folder')

/bin/sh: module: command not found
/bin/sh: module: command not found
Python 2.7.5
/bin/python
/global/software/sl-7.x86_64/modules/langs/python/3.5/bin/python3
Python 3.5.4 :: Anaconda custom (64-bit)
/global/software/sl-7.x86_64/modules/langs/python/3.5/bin/jupyter
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
/global/home/users/myashar/.conda/envs/py3.5/bin/python
3.5.5 |Anaconda, Inc.| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]
sys.version_info(major=3, minor=5, micro=5, releaselevel='final', serial=0)
user name:  myashar


In [2]:
import sys
print(sys.executable)
print(sys.path)
print(sys.version)
# !conda list

/global/home/users/myashar/.conda/envs/py3.5/bin/python
['', '/global/home/users/myashar/.conda/envs/py3.5/lib/python35.zip', '/global/home/users/myashar/.conda/envs/py3.5/lib/python3.5', '/global/home/users/myashar/.conda/envs/py3.5/lib/python3.5/plat-linux', '/global/home/users/myashar/.conda/envs/py3.5/lib/python3.5/lib-dynload', '/global/home/users/myashar/.local/lib/python3.5/site-packages', '/global/home/users/myashar/.conda/envs/py3.5/lib/python3.5/site-packages', '/global/home/users/myashar/.local/lib/python3.5/site-packages/IPython/extensions', '/global/home/users/myashar/.ipython']
3.5.5 |Anaconda, Inc.| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]


## Project set up
After you run the code box below, a set of text boxes will appear. Click in the text boxes and enter the following values:
* **Project name**: The name of your project. Will be used for some of the folders on the cluster that will be created to process the project.
* **Mount folder**: Where your photos will be uploaded for processing. Unless you know specifically this should be somewhere else, leave this as /scratch/
* **Box project folder**: The name of the folder on Box that has your photos in it.
* **Path to container**: The location on the cluster where the container with the Photoscan software is installed. At Berkeley, this container is at /global/scratch/groups/dh/photoscan13.img --> photoscan_1_4_3.simg
* **Home folder**: Your home folder on the cluster. 
* **Run folder**: The folder where the various job scripts to trigger the processing steps will be stored. Default: /global/scratch/your-user-name/your-project-name/

**MYashar: Make sure that the latest version of the Chrome or Safari browser is being used; otherwise the widget 
text boxes and the sliders (later on), may not work.**

When you're done filling in the text boxes, hit the "run cell" button again.


In [3]:
# Enter the projectname
interact(pn, projectName=pntextbox)

# Enter the Singularity Mount Folder here, this is defined inside the Singularity container:
interact(smf, mountFolder=singularitymountfoldertextbox)

# Enter the Box Project folder
interact(bpf, boxFolder=boxfoldertextbox)

# MYashar -- maybe add a part here: Enter the Box Image folder
# interact(bif, boxImageFolder=boximagetextbox)   (?)

# Enter the path to the Singularity container hosting Photoscan
interact(scp, containerPath=scptextbox)

# Enter the path to where the iamge files from Box should be placed on the scrath drive
# interact(idd, imageDataPath=iddtextbox)

# Enter the path to the user home folder where Box oauth information is stored in files.
interact(hfp, homePath=hometextbox)

# Enter the path to the run home folder where processing will be conducted and intermediate files created.
interact(rf, runPath=runtextbox)

interactive(children=(Text(value='projectname', description='Project Name'), Output()), _dom_classes=('widget-…

interactive(children=(Text(value='/scratch/', description='mountFolder'), Output()), _dom_classes=('widget-int…

interactive(children=(Text(value='myBoxFolderName', description='Box Project Folder'), Output()), _dom_classes…

interactive(children=(Text(value='/global/scratch/myashar/container/photoscan_1_4_3.simg', description='Path t…

interactive(children=(Text(value='/global/home/users/myashar/', description='Home Folder'), Output()), _dom_cl…

interactive(children=(Text(value='/global/scratch/myashar/projectname/', description='Run Folder'), Output()),…

<function __main__.rf(runPath)>

## Creating project folders
The code box below creates the project folders (e.g. run folder, scratch images directory) on the cluster if they don't already exist.

In [4]:
projectname =  pntextbox.value

# Defines all paths and filenames
import os

usernamearray = %sx whoami
username = usernamearray[0]
print('user name: ', username)
print('project name: ', projectname)

runFolder = os.path.join(runtextbox.value, '')
homeFolder =  os.path.join(hometextbox.value, '')
print('run folder: ', runFolder)
print('home folder: ', homeFolder)

# Creates the run folder if it does not exist
if ( not os.path.isdir(runFolder)):
    print('run folder does not exist, creating.')
    os.mkdir(runFolder)

singularitymountfolder = os.path.join(singularitymountfoldertextbox.value, '')
print('Singulariy Mount Folder: ', singularitymountfolder)
boxProjectFolder = boxfoldertextbox.value
projectFile = '/' + projectname + '.psx'
datazipFile = '/' + projectname + '.files.zip'
dataFolder = '/' + projectname + '.files'
print('Box Project Folder: ',boxProjectFolder)
print('project file: ', projectFile)
print('data zip file: ', datazipFile)
print('data folder: ', dataFolder)

projectfileid = ''
jpgFile = projectname + '.jpg'
print('jpg file: ',jpgFile)

scratchImageDataDirectory = runFolder + 'images/'
print('Scratch Image Data Directory: ', scratchImageDataDirectory)
# Creates the image folder if it does not exist
if ( not os.path.isdir(scratchImageDataDirectory)):
    print('images folder does not exist, creating.')
    os.mkdir(scratchImageDataDirectory)

slurmScript = runFolder + 'slurmscript.sh'
execScript = runFolder + 'execscript.sh'
scratchExecScript = singularitymountfolder + 'execscript.sh'
commandScript = runFolder + 'commandscript.sh'
print('Slurm script: ', slurmScript)
print('exec script: ', execScript)
print('scratch exec script: ', scratchExecScript)
print('command script: ', commandScript)


singularityContainerPath = scptextbox.value
#/global/scratch/groups/dh/photoscan_1_4_3.simg
print("Singularity Container Path", singularityContainerPath)

user name:  myashar
project name:  projectname
run folder:  /global/scratch/myashar/projectname/
home folder:  /global/home/users/myashar/
Singulariy Mount Folder:  /scratch/
Box Project Folder:  myBoxFolderName
project file:  /projectname.psx
data zip file:  /projectname.files.zip
data folder:  /projectname.files
jpg file:  projectname.jpg
Scratch Image Data Directory:  /global/scratch/myashar/projectname/images/
Slurm script:  /global/scratch/myashar/projectname/slurmscript.sh
exec script:  /global/scratch/myashar/projectname/execscript.sh
scratch exec script:  /scratch/execscript.sh
command script:  /global/scratch/myashar/projectname/commandscript.sh
Singularity Container Path /global/scratch/myashar/container/photoscan_1_4_3.simg


## Box setup

To connect Box with this notebook, you need to go through some one-time configuration steps the first time you use this notebook for any project. Please run the *BoxAuthenticationBootstrap.ipynb* notebook, then return to the code cells below to finish setup. After you finish the last cell, you should see *Folder name:  All Files*.

If you run this notebook again for another project, you won't have to run *BoxAuthenticationBootstrap.ipynb* again, and can just continue to the code below.

In [27]:
# Might want to make sure that *.cfg files that were created in earlier runs are copied to home directory.
!cp *.cfg ~/

In [28]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open(homeFolder,'apptoken.cfg', 'w') as f:
     f.write(refresh_token.strip())

In [29]:
!pwd

/global/scratch/myashar/projectname/images


In [38]:
import os

CLIENT_ID = None
CLIENT_SECRET = None
REDIRECT_URI = None

# CLIENT_ID = '7fv9qix8gzo77vhwidetl426az41xneg'
# CLIENT_SECRET = 'agzSPeOv8Zmmj8iupLJFcl9lWolaBqtY'
# REDIRECT_URI ='https://berkeley.app.box.com/folder/0'

# Read app info from text file
# with open(homeFolder +'app.cfg', 'r') as app_cfg:
with open('app.cfg', 'r') as app_cfg:    
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    REDIRECT_URI = app_cfg.readline()


In [39]:
REFRESH_TOKEN = None

# Read app info from text file
# with open(homeFolder + 'apptoken.cfg', 'r') as apptoken_cfg:
with open('apptoken.cfg', 'r') as apptoken_cfg:
    REFRESH_TOKEN = apptoken_cfg.readline()

In [40]:
from boxsdk import OAuth2
from boxsdk import Client
!pwd
# Do OAuth2 authorization.
oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    refresh_token=REFRESH_TOKEN.strip(),
    store_tokens=store_tokens
)

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("Folder name: ", root_folder['name'] )

items = client.folder(folder_id='0').get_items(limit=100, offset=0)
print ("items: ", items )
!pwd


/global/scratch/myashar/projectname/images


TypeError: an integer is required (got type str)

## Finding folder on box
The function below looks for your project folder on Box.

In [41]:
def find_folder_id(folder_name):
    print ('find_folder_id folder_name: ', folder_name)
    folderlist = client.search(query=folder_name, result_type='folder', limit=100, offset=0)
    
    print ('find_folder_id folderlist: ', folderlist)
    
    if len(folderlist) == 0:
        print('folder not found: ', folder_name)
        return 0
    else:
        for fldr in folderlist:
            if fldr['name'] == folder_name :
                return fldr['id']
            
        return 0


## Create command scripts template
This code writes out a execute file that includes the license install as the first step each time to enable any HPC node. This is a temporary workaround until release 1.3 is installed on Savio and a license server is configured.

In [42]:
# script for singularity to run
import os, stat
os.system('export XAUTHORITY=/global/home/users/myashar/.Xauthority')
os.system('export DISPLAY=:0')
!module load qt
os.system("module load qt")

# execScriptTemplate = 'export RLM_LICENSE=5053@lmgr0.brc.berkeley.edu \n\
# /opt/photoscan-pro/photoscan.sh -r  /scratch/commandscript.sh \n'

# I needed to run the following for my particular case instead:
execScriptTemplate = 'export RLM_LICENSE=5053@lmgr0.brc.berkeley.edu \n\
/usr/local/photoscan-pro/photoscan.sh -r  /scratch/commandscript.sh \n'

with open(execScript, 'w') as f:
    f.write(execScriptTemplate)
    
os.chmod(execScript,  0o755)    


/bin/sh: module: command not found


In [43]:
# batch script
os.system('export XAUTHORITY=/global/home/users/myashar/.Xauthority')
os.system('export DISPLAY=:0')
# !module load qt
os.system("module load qt")
os.environ['QT_QPA_PLATFORM']='offscreen'

batchtemplate = '#!/bin/bash  \n\
# Job name: \n\
#SBATCH --job-name=' + projectname + '\n\
# \n\
# Account: \n\
#SBATCH --account=ac_scsguest \n\
# \n\
# Partition: \n\
#SBATCH --partition=savio \n\
# \n\
# Wall clock limit: \n\
#SBATCH --time={} \n\
# \n\
## Command(s) to run: \n\
## source activate py3.5 \n\

singularity -d exec -B ' + runFolder + ':' + singularitymountfolder + '  ' + singularityContainerPath + '  ' + scratchExecScript + ' -platform offscreen \n' 
!export DISPLAY=':0.0'
!export XAUTHORITY=/global/home/users/myashar/.Xauthority
!export DISPLAY=:0

## Function for updating project file
Multiple times in the workflow, the project folder needs to be updated on Box in order to enable making edits through the desktop version of Photoscan. This sets up a utility function that pushes the project 
file into the working directory on Savio. It assumes that the project folder will only contain one project file and one zipped archive.

In [44]:
import os
import shutil

def update_project_file_in_box():

    newFolderId = find_folder_id(boxProjectFolder)
    print ("Box folder id: ", newFolderId  )
    
    # create a zipped archive of the data folder
    shutil.make_archive(runFolder + dataFolder, 'zip', runFolder + dataFolder)
    
    projectfilelist = client.search(query=projectFile, result_type='file', limit=10, 
                                offset=0, ancestor_folders=[client.folder(folder_id=newFolderId)],
                                file_extensions=['psx'] )  
    
    print ("project file list: ", projectfilelist  )
    
    # if project file is not yet in folder, upload both the project file and the data zip file
    if len(projectfilelist) == 0 or len(projectfilelist) > 1: 
        print ("project file not found. " )
        upload_folder = client.folder(folder_id=newFolderId).get()
        print("upload_folder: ", upload_folder)
        projectpsx = upload_folder.upload(runFolder + projectFile)  
        print ("project file id: ", projectpsx['id'] )
        projectzip = upload_folder.upload(runFolder + datazipFile)
        print("projectzip: ", projectzip)
        print ("data zip file id: ", projectzip['id'] )
        return
    else:
        datazipfilelist = client.search(query=datazipFile, result_type='file', limit=10, 
                                offset=0, ancestor_folders=[client.folder(folder_id=newFolderId)])
                                #file_extensions=['zip'] )  
        print ("datazip file list: ", datazipfilelist  )
        
        projectfileid = projectfilelist[0]['id']
        print ("project file id: ", projectfileid )
        datazipfileid = datazipfilelist[0]['id']
        print ("data zip file list: ", datazipfileid  )
    
        update_file = client.file(file_id=projectfileid).get()
        update_zip_file = client.file(file_id=datazipfileid).get()

        # upload a new version of the project file
        print ('begin project file update.' )
        psxfile = update_file.update_contents(runFolder + projectFile)  
        zipfile = update_zip_file.update_contents(runFolder + datazipFile)
        print ('update psx result: ', psxfile ,'   update zip result: ', zipfile )

## Fetching images from Box
The code below retrieves the images from your project directory on Box, and puts them in the project folder on the cluster that you specified above.

When you run it, it will check for the number of files in the folder on Box, and list each of them as it downloads. Please wait until you see *Download complete* before moving to the next step -- this can take some time if you have a lot of images, or large images.

In [45]:
import os
import shutil 
print("boxProjectFolder:",boxProjectFolder)

os.chdir(scratchImageDataDirectory)
print ('retrieving images from folder: ', boxProjectFolder)
print ('downloading images to directory: ', scratchImageDataDirectory)

# test folder contents
items = client.folder(folder_id='0').get_items(limit=25, offset=0)
if type(items) is list:
    print ('number of files in top folder: ', len(items) )
    
    targetfolderId = ''
    for item in items:
        if item['type'] == 'folder':
            print('folder name: ', item['name'])
            if item['name'] == boxProjectFolder:
                targetfolderId = item['id']
                print('targetfolderId: ', targetfolderId)
        
    if targetfolderId is not None:
        tgtitems = client.folder(folder_id=targetfolderId).get_items(limit=200, offset=0)
        if type(tgtitems) is list:
            print ('number of files in target folder: ', len(tgtitems) ) 
        
        # download all image files
        for tgtitem in tgtitems:
            if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.JPG'):
                # If necessary, remember to change '.JPG' to '.jpg' here
                print('downloading: ', tgtitem['name'])
                imagecontent = client.file(file_id=tgtitem['id']).content()
                newfile = open(scratchImageDataDirectory + tgtitem['name'], 'wb')
                newfile.write(imagecontent)
                newfile.close()
        print ('Download complete')

boxProjectFolder: myBoxFolderName
retrieving images from folder:  myBoxFolderName
downloading images to directory:  /global/scratch/myashar/projectname/images/
number of files in top folder:  7
folder name:  Astronomy-Microscopy Summit
folder name:  My Box Notes
folder name:  myBoxFolderName
targetfolderId:  53223729428
folder name:  myBoxFolderName_Test1
folder name:  savio_tests
folder name:  test_data
folder name:  workshop
number of files in target folder:  46
downloading:  IMG_0324.JPG
downloading:  IMG_0325.JPG
downloading:  IMG_0326.JPG
downloading:  IMG_0327.JPG
downloading:  IMG_0328.JPG
downloading:  IMG_0329.JPG
downloading:  IMG_0330.JPG
downloading:  IMG_0331.JPG
downloading:  IMG_0332.JPG
downloading:  IMG_0333.JPG
downloading:  IMG_0334.JPG
downloading:  IMG_0335.JPG
downloading:  IMG_0336.JPG
downloading:  IMG_0337.JPG
downloading:  IMG_0338.JPG
downloading:  IMG_0339.JPG
downloading:  IMG_0340.JPG
downloading:  IMG_0341.JPG
downloading:  IMG_0342.JPG
downloading:  IMG_

## Function for retrieving project file
This code sets up a function to bring an updated project file back from Box, e.g. after you've made changes using the desktop version of Photoscan, and you want to run a subsequent processing step on a cluster.

In [46]:

import os
import shutil 
def retrieve_project_file():
    newFolderId = find_folder_id(boxProjectFolder)
    tgtitems = client.folder(folder_id=newFolderId).get_items(limit=1000, offset=0)

    # download the project file
    for tgtitem in tgtitems:
        if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.psx'):
            print('downloading: ', tgtitem['name'])
            imagecontent = client.file(file_id=tgtitem['id']).content()
            newfile = open(runFolder + tgtitem['name'], 'wb')
            newfile.write(imagecontent)
            newfile.close()
            print("new file: ", newfile)
            print('project file download complete.')
            
        if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.zip'):
            
            # delete the old folder
            !rm -rf $runFolder$dataFolder
            
            print('downloading: ', tgtitem['name'])
            imagecontent = client.file(file_id=tgtitem['id']).content()
            newfile = open(runFolder + tgtitem['name'], 'wb')
            newfile.write(imagecontent)
            newfile.close()
            shutil.unpack_archive(runFolder + datazipFile, runFolder + dataFolder ,'zip')
            
            print('data zip file download complete.')

# Project setup

This section pulls your images onto the computing cluster and creates and uploads a Photoscan .psx file if it doesn't already exist.



## Set up job script

This code sets up the job script that will be run on the cluster, in order to pull in your images.

In [47]:
def hrs(xhrs):
    return xhrs
def mins(xmins):
    return xmins

#Define widgets and defaults
hrslider=widgets.IntSlider(description='Estimated Hours:',  value=1, min=0, max=23,step=1)
minslider=widgets.IntSlider(description='Estimated Minutes:',  value=0, min=0, max=59,step=1)


In [50]:
# Enter the estimated hours and minutes to create the project and add photos
interact(hrs, xhrs=hrslider)
interact(mins, xmins=minslider)


interactive(children=(IntSlider(value=0, description='Estimated Hours:', max=23), Output()), _dom_classes=('wi…

interactive(children=(IntSlider(value=2, description='Estimated Minutes:', max=59), Output()), _dom_classes=('…

<function __main__.mins(xmins)>

In [51]:
import os, stat
os.system('export XAUTHORITY=/global/home/users/myashar/.Xauthority')
os.system('export DISPLAY=:0')
os.system("module load qt")
os.environ['QT_QPA_PLATFORM']='offscreen'
files = [ singularitymountfolder + 'images/' + f for f in os.listdir(scratchImageDataDirectory) ] 

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
doc = PhotoScan.app.document \n\
chunk = PhotoScan.app.document.addChunk() \n\
chunk.addPhotos( {} ) \n\
doc.save(path=\"{}\", chunks = [doc.chunk])\n '
!export DISPLAY=':0.0'
!export XAUTHORITY=/global/home/users/myashar/.Xauthority
!export DISPLAY=:0
output = template.format(str(files), singularitymountfolder + projectFile)
print('output: ', output)

with open(commandScript, 'w') as f:  
    f.write(output)

os.chmod(commandScript,  0o755)

#set time limit for this batch run
minstring = str(minslider.value)
if minslider.value < 10:
    minstring = '0' + str(minslider.value)
estimateTime = str(hrslider.value) + ':' + minstring + ':00'
print('estimateTime: ', estimateTime)
outputbatchscript = batchtemplate.format(estimateTime)
with open(slurmScript, 'w') as f:  
    f.write(outputbatchscript)
    
os.chmod(slurmScript,  0o755)

output:  #!/usr/bin/env python3 
import PhotoScan 
import time 
doc = PhotoScan.app.document 
chunk = PhotoScan.app.document.addChunk() 
chunk.addPhotos( ['/scratch/images/IMG_0338.JPG', '/scratch/images/IMG_0325.JPG', '/scratch/images/IMG_0333.JPG', '/scratch/images/IMG_0329.JPG', '/scratch/images/IMG_0330.JPG', '/scratch/images/IMG_0334.JPG', '/scratch/images/IMG_0337.JPG', '/scratch/images/IMG_0346.JPG', '/scratch/images/IMG_0327.JPG', '/scratch/images/IMG_0336.JPG', '/scratch/images/apptoken.cfg', '/scratch/images/IMG_0324.JPG', '/scratch/images/IMG_0340.JPG', '/scratch/images/IMG_0342.JPG', '/scratch/images/app.cfg', '/scratch/images/IMG_0339.JPG', '/scratch/images/IMG_0335.JPG', '/scratch/images/IMG_0331.JPG', '/scratch/images/IMG_0328.JPG', '/scratch/images/IMG_0326.JPG', '/scratch/images/IMG_0341.JPG', '/scratch/images/IMG_0332.JPG'] ) 
doc.save(path="/scratch//projectname.psx", chunks = [doc.chunk])
 
estimateTime:  0:02:00


## Run job
The code below will run the job on the cluster, pulling in your images from Box, and will display the job ID.

In [52]:
os.system('export XAUTHORITY=/global/home/users/myashar/.Xauthority')
os.system('export DISPLAY=:0')
os.environ['QT_QPA_PLATFORM']='offscreen'
os.system("module load qt")
os.chdir(runFolder)
print(runFolder)
out = !sbatch slurmscript.sh   
print(out)   
print ('Execute image loading output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)

/global/scratch/myashar/projectname/
['Submitted batch job 4807120']
Execute image loading output:  ['Submitted batch job 4807120']
4807120


In [53]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4807120     savio projectn  myashar PD       0:00      1 (Resources)
--------------------------------
JobId=4807120 JobName=projectname
   UserId=myashar(43974) GroupId=ucb(501) MCS_label=N/A
   Priority=377307 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=PENDING Reason=Resources Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:00:00 TimeLimit=00:02:00 TimeMin=N/A
   SubmitTime=2019-07-18T16:51:29 EligibleTime=2019-07-18T16:51:29
   StartTime=2019-07-18T16:53:07 EndTime=2019-07-18T16:55:07 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   LastSchedEval=2019-07-18T16:53:04
   Partition=savio AllocNode:Sid=jupyter:425871
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=(null) SchedNodeList=n0027.savio1
   NumNodes=1 NumCPUs=1 NumTasks=1 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=1,mem=62.50G,node=1
   Socks/Node=* Ntas

## Upload project file
If there isn't a "data" subfolder inside your project folder on Box already, the code below will first create it. The code will then upload a new Photoscan project file and associated data files to Box. 

**MYashar: As far as things stand now with the code, no such "data" subfolder is created on Box.**

If a Photoscan project file with the same name already exists in the folder on Box, this code will *not* overwrite it.

Please note: New folders in Box can take several minutes to register, so this step may take a few minutes. If you get a "file not found" error in the next step, wait a minute or two and retry.

In [78]:
from boxsdk.exception import BoxAPIException

newFolderId = find_folder_id(boxProjectFolder)
print('folder id: ', newFolderId)

if newFolderId == 0:
    newFolder = client.folder(folder_id='0').create_subfolder(boxProjectFolder)
    newFolderId = newFolder['id']
    print ("folder created" )
else: 
    print ("folder exists" )
    
print ("folder id: ", newFolderId )
# print("new folder: ", newFolder)
update_project_file_in_box()


find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
folder id:  53223729428
folder exists
folder id:  53223729428
find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
Box folder id:  53223729428
project file list:  [<Box File - 317022687393 (projectname.psx)>]
datazip file list:  [<Box File - 317017752277 (projectname.files.zip)>, <Box File - 319186364581 (README.docx)>, <Box File - 319043117371 (slurm-generate_model_output.txt)>, <Box File - 319044387168 (slurm-build_texture.txt)>, <Box File - 319044323866 (slurm-export_photomosaic_format_output.txt)>, <Box File - 319035518579 (slurm-export_dense_cloud.txt)>, <Box File - 319043284628 (slurm-pull_images_from_Box.txt)>, <Box File - 319043305927 (slurm-build_dense_cloud.txt)>, <Box File - 319041408570 (

## OFFLINE: Image masking
If you need to mask any images before building the dense cloud, please launch your desktop version of Photoscan, make those changes, and save the modified .psx file back to your folder on Box before proceeding with the next step.

# Align photos and Build Dense Cloud

The code below will retrieve the project file from Box. This will ensure that you're working with the latest version, even if you've made changes offline (such as image masking).

In [79]:
retrieve_project_file()

find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
downloading:  projectname.files.zip
data zip file download complete.
downloading:  projectname.psx
new file:  <_io.BufferedWriter name='/global/scratch/myashar/projectname/projectname.psx'>
project file download complete.


## Set up job script

Run the code box below, then choose your estimated run time for the job. The default run time is 3 hours, which should cover most projects with under 75 photos. If you have more photos, please increase the run time.

**Maurice: make this a different widget with default run time of 3 hours**

In [58]:
# Enter the estimated hours and minutes to create the project and add photos
interact(hrs, xhrs=hrslider)
interact(mins, xmins=minslider)

interactive(children=(IntSlider(value=0, description='Estimated Hours:', max=23), Output()), _dom_classes=('wi…

interactive(children=(IntSlider(value=2, description='Estimated Minutes:', max=59), Output()), _dom_classes=('…

<function __main__.mins(xmins)>

## Set up parameters
Choose the quality settings for the dense cloud build after running the cells below.

In [59]:
def chc(txt):
    return txt

pselchoice=widgets.Dropdown(description='Preselection', value='GenericPreselection', options=['GenericPreselection', 'ReferencePreselection'])
qualchoice=widgets.Dropdown(description='Quality', value='MediumQuality', options=['UltraQuality', 'HighQuality', 'MediumQuality',  'LowQuality', 'LowestQuality'] )
fltrchoice=widgets.Dropdown(description='Filter', value='ModerateFiltering', options=['NoFiltering', 'MildFiltering', 'ModerateFiltering',  'AggressiveFiltering']  )
accchoice=widgets.Dropdown(description='Accuracy', value='MediumAccuracy', options= ['HighestAccuracy', 'HighAccuracy', 'MediumAccuracy',  'LowAccuracy', 'LowestAccuracy']  )
afchoice=widgets.Dropdown(description='Adaptive Fitting', options=['True', 'False'])

In [60]:
interact(chc, txt=pselchoice)
interact(chc, txt=qualchoice)
interact(chc, txt=fltrchoice)
interact(chc, txt=accchoice)
interact(chc, txt=afchoice)

interactive(children=(Dropdown(description='Preselection', options=('GenericPreselection', 'ReferencePreselect…

interactive(children=(Dropdown(description='Quality', index=2, options=('UltraQuality', 'HighQuality', 'Medium…

interactive(children=(Dropdown(description='Filter', index=2, options=('NoFiltering', 'MildFiltering', 'Modera…

interactive(children=(Dropdown(description='Accuracy', index=2, options=('HighestAccuracy', 'HighAccuracy', 'M…

interactive(children=(Dropdown(description='Adaptive Fitting', options=('True', 'False'), value='True'), Outpu…

<function __main__.chc(txt)>

## Create job script
The next cell will create and display the job script that will be run on the cluster.

In [80]:
# Creates job script for building dense cloud

import os, stat

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.matchPhotos(accuracy=PhotoScan.{}, preselection=PhotoScan.{}) \n\
chunk.alignCameras(adaptive_fitting={}) \n\
## In Photoscan version 1.4.* the dense cloud generation task has been split into two parts - \n\
## depth maps generation and dense cloud generation:   \n\
chunk.buildDepthMaps(quality=PhotoScan.{}, filter=PhotoScan.{} ) \n\
chunk.buildDenseCloud(point_colors = True) \n\
doc.save(\"{}\") \n\
print( "stop time: ", time.time()) \n'
         
output = template.format(singularitymountfolder + projectFile, accchoice.value, pselchoice.value, afchoice.value, qualchoice.value, fltrchoice.value, singularitymountfolder + projectFile)

print('output: ', output)

with open(commandScript, 'w') as f:  
    f.write(output)

    
#Set time limit for this batch run
minstring = str(minslider.value)
if minslider.value < 10:
    minstring = '0' + str(minslider.value)
estimateTime = str(hrslider.value) + ':' + minstring + ':00'
print('estimateTime: ', estimateTime)
outputbatchscript = batchtemplate.format(estimateTime)
#outputbatchscript = batchtemplate.format('01:30:00')
with open(slurmScript, 'w') as f:  
    f.write(outputbatchscript)
    

output:  #!/usr/bin/env python3 
import PhotoScan 
import time 
print( "start time: ", time.time()) 
doc = PhotoScan.app.document 
doc.open("/scratch//projectname.psx") 
chunk = doc.chunk 
chunk.matchPhotos(accuracy=PhotoScan.MediumAccuracy, preselection=PhotoScan.GenericPreselection) 
chunk.alignCameras(adaptive_fitting=True) 
## In Photoscan version 1.4.* the dense cloud generation task has been split into two parts - 
## depth maps generation and dense cloud generation:   
chunk.buildDepthMaps(quality=PhotoScan.MediumQuality, filter=PhotoScan.ModerateFiltering ) 
chunk.buildDenseCloud(point_colors = True) 
doc.save("/scratch//projectname.psx") 
print( "stop time: ", time.time()) 

estimateTime:  0:05:00


## Run dense cloud build
Running the following code box will submit your job to the cluster. Your job ID will be printed below.

In [81]:
os.system('export XAUTHORITY=/global/home/users/myashar/.Xauthority')
os.system('export DISPLAY=:0')
os.environ['QT_QPA_PLATFORM']='offscreen'
os.system("module load qt")
os.chdir(runFolder)
print(runFolder)
out = !sbatch slurmscript.sh   
    
print ('Execute the dense cloud build output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)

/global/scratch/myashar/projectname/
Execute the dense cloud build output:  ['Submitted batch job 4810073']
4810073


## Check on job status
Building the dense cloud is the longest-running phase of the photogrammetry process. Depending on how many photos you have, and the quality settings you've chosen, the job could run for anywhere from minutes to hours. Run the following code box to check on the status of the dense cloud build. You can run it multiple times for new updates.

In [86]:
# print the users queue and the job status by id
!squeue -u $username
!squeue -u $USER
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
--------------------------------
JobId=4810073 JobName=projectname
   UserId=myashar(43974) GroupId=ucb(501) MCS_label=N/A
   Priority=381693 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=COMPLETED Reason=None Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:01:26 TimeLimit=00:05:00 TimeMin=N/A
   SubmitTime=2019-07-19T19:18:54 EligibleTime=2019-07-19T19:18:54
   StartTime=2019-07-19T20:05:03 EndTime=2019-07-19T20:06:29 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   LastSchedEval=2019-07-19T20:05:03
   Partition=savio AllocNode:Sid=jupyter:425871
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0062.savio1
   BatchHost=n0062.savio1
   NumNodes=1 NumCPUs=20 NumTasks=0 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1,billing=20


## Display log message
As an optional step, you can run the code below to see the log file once the job is done. It includes technical information about the points that were found in each photo, rectifying disparities between photos, etc.

In [87]:
# print the log file if necessary
slurmlogfilename = 'slurm-' + jobId + '.out'
with open(slurmlogfilename, 'r') as fin:
    print(fin.read(), end="")

Enabling debugging
Ending argument loop
Singularity version: 2.6-dist
Exec'ing: /usr/libexec/singularity/cli/exec.exec
Evaluating args: '-B /global/scratch/myashar/projectname/:/scratch/ /global/scratch/myashar/container/photoscan_1_4_3.simg /scratch/execscript.sh -platform offscreen'
VERBOSE [U=43974,P=85427]  message_init()                            Set messagelevel to: 5
VERBOSE [U=43974,P=85427]  singularity_config_parse()                Initialize configuration file: /etc/singularity/singularity.conf
DEBUG   [U=43974,P=85427]  singularity_config_parse()                Starting parse of configuration file /etc/singularity/singularity.conf
VERBOSE [U=43974,P=85427]  singularity_config_parse()                Got config key allow setuid = 'yes'
VERBOSE [U=43974,P=85427]  singularity_config_parse()                Got config key max loop devices = '256'
VERBOSE [U=43974,P=85427]  singularity_config_parse()                Got config key allow pid ns = 'yes'
VERBOSE [U=43974,P=85427]  si

## Export dense cloud
If you want to export the dense cloud for use in other software, you can run the code boxes below. If you want to just continue with the next step in the photogrammetry process, you can skip it.

In [88]:
# Quinn - Im guessing at these extenstions, I have only used oc3
pointsFormatExt = { 'PointsFormatOBJ':'.obj', 'PointsFormatPLY':'.ply', 'PointsFormatXYZ':'.xyz', 'PointsFormatLAS':'.las', 'PointsFormatExpe':'.???',
'PointsFormatU3D':'.u3d', 'PointsFormatPDF':'.pdf', 'PointsFormatE57':'.e57', 'PointsFormatOC3':'.oc3', 'PointsFormatPotree':'.???',
'PointsFormatLAZ':'.laz', 'PointsFormatCL3':'.cl3', 'PointsFormatPTS':'.pts', 'PointsFormatDXF':'.dxf' }

ptsfchoice=widgets.Dropdown(description='Points Format', value='PointsFormatOC3', options= ['PointsFormatOBJ', 'PointsFormatPLY', 'PointsFormatXYZ', 'PointsFormatLAS', 'PointsFormatExpe',
'PointsFormatU3D', 'PointsFormatPDF', 'PointsFormatE57', 'PointsFormatOC3', 'PointsFormatPotree',
'PointsFormatLAZ', 'PointsFormatCL3', 'PointsFormatPTS', 'PointsFormatDXF']   )


In [89]:
interact(chc, txt=ptsfchoice)

interactive(children=(Dropdown(description='Points Format', index=8, options=('PointsFormatOBJ', 'PointsFormat…

<function __main__.chc(txt)>

In [90]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportPoints(path=\"{}\", format=PhotoScan.PointsFormat.{})  \n' 

output = template.format(singularitymountfolder + projectFile,  singularitymountfolder + 'photoscandemo' + pointsFormatExt[ptsfchoice.value], ptsfchoice.value)

print('output:', output)

with open(commandScript, 'w') as f:
    f.write(output)

output: #!/usr/bin/env python3 
import PhotoScan 
doc = PhotoScan.app.document 
doc.open("/scratch//projectname.psx") 
chunk = doc.chunk 
chunk.exportPoints(path="/scratch/photoscandemo.oc3", format=PhotoScan.PointsFormat.PointsFormatOC3)  



In [91]:
out = !sbatch slurmscript.sh   
        
print (' Export the points  output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)

 Export the points  output:  ['Submitted batch job 4810117']
4810117


In [92]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
--------------------------------
JobId=4810117 JobName=projectname
   UserId=myashar(43974) GroupId=ucb(501) MCS_label=N/A
   Priority=380401 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=COMPLETED Reason=None Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:00:04 TimeLimit=00:05:00 TimeMin=N/A
   SubmitTime=2019-07-19T20:09:53 EligibleTime=2019-07-19T20:09:53
   StartTime=2019-07-19T20:09:54 EndTime=2019-07-19T20:09:58 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   LastSchedEval=2019-07-19T20:09:54
   Partition=savio AllocNode:Sid=jupyter:425871
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0159.savio1
   BatchHost=n0159.savio1
   NumNodes=1 NumCPUs=20 NumTasks=0 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1,billing=20
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPUsNode=1 MinMemoryNode=62

## Update project on Box
After generating the dense cloud, the .psx file will be much larger and this may take a moment.
**Maurice: add success message**

In [94]:
update_project_file_in_box()

find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
Box folder id:  53223729428
project file list:  [<Box File - 317022687393 (projectname.psx)>]
datazip file list:  [<Box File - 317017752277 (projectname.files.zip)>, <Box File - 319186364581 (README.docx)>, <Box File - 319043117371 (slurm-generate_model_output.txt)>, <Box File - 319044387168 (slurm-build_texture.txt)>, <Box File - 319044323866 (slurm-export_photomosaic_format_output.txt)>, <Box File - 319035518579 (slurm-export_dense_cloud.txt)>, <Box File - 319043284628 (slurm-pull_images_from_Box.txt)>, <Box File - 319043305927 (slurm-build_dense_cloud.txt)>, <Box File - 319041408570 (slurm-build_mesh.txt)>]
project file id:  317022687393
data zip file list:  317017752277
begin project file update.
update psx result:  <Box File - 317022687393>    update zip result:  <Box File - 317017752277>


## OFFLINE: Dense cloud clean-up
At this point, you may want to clean up the dense point cloud by removing stray points (e.g. for objects that have not been photographed in a lightbox, where there might be artifacts from the background, etc.) This should be done using the desktop version of Photoscan. 

Be sure to save the updated project file back to the same place in Box if you want to run the next processing steps through this notebook.

# Build Mesh

First, retrieve the project file (.psx) from Box, in case it has been updated.

In [96]:
retrieve_project_file()

find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
downloading:  projectname.files.zip
data zip file download complete.
downloading:  projectname.psx
new file:  <_io.BufferedWriter name='/global/scratch/myashar/projectname/projectname.psx'>
project file download complete.


## Choose parameters
Running the code box below will create three sets of drop-downs where you can choose the surface type, face count, and data source.

In [97]:
stchoice=widgets.Dropdown(description='Surface type', value='Arbitrary', options=['Arbitrary', 'HeightField'])
fcchoice=widgets.Dropdown(description='Face count', value='HighFaceCount', options= ['LowFaceCount', 'MediumFaceCount', 'HighFaceCount'] )
dschoice=widgets.Dropdown(description='Data source', value='DenseCloudData', options=['PointCloudData', 'DenseCloudData', 'DepthMapsData', 'ModelData', 'TiledModelData', 'ElevationData',
'OrthomosaicData']  )


In [98]:
interact(chc, txt=stchoice)
interact(chc, txt=fcchoice)
interact(chc, txt=dschoice)

interactive(children=(Dropdown(description='Surface type', options=('Arbitrary', 'HeightField'), value='Arbitr…

interactive(children=(Dropdown(description='Face count', index=2, options=('LowFaceCount', 'MediumFaceCount', …

interactive(children=(Dropdown(description='Data source', index=1, options=('PointCloudData', 'DenseCloudData'…

<function __main__.chc(txt)>

In [99]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildModel(surface=PhotoScan.{}, source=PhotoScan.{}, face_count=PhotoScan.{} ) \n\
doc.save(\"{}\") \n\
print( "stop time: ", time.time()) \n'

output = template.format(singularitymountfolder + projectFile, stchoice.value, dschoice.value, fcchoice.value, singularitymountfolder + projectFile)

print('output:', output)

with open(commandScript, 'w') as f:  
    f.write(output)


#set time limit for this batch run
outputbatchscript = batchtemplate.format('00:30:00')
with open(slurmScript, 'w') as f:  
    f.write(outputbatchscript)
    

output: #!/usr/bin/env python3 
import PhotoScan 
import time 
print( "start time: ", time.time()) 
doc = PhotoScan.app.document 
doc.open("/scratch//projectname.psx") 
chunk = doc.chunk 
chunk.buildModel(surface=PhotoScan.Arbitrary, source=PhotoScan.DenseCloudData, face_count=PhotoScan.HighFaceCount ) 
doc.save("/scratch//projectname.psx") 
print( "stop time: ", time.time()) 



## Run mesh build
The code box below will submit your job and print out your job ID.

In [100]:
out = !sbatch slurmscript.sh   
    
print ('Execute the mesh build output: ', out ) 
jobId =  out[0].split()[3]
print (jobId) 

Execute the mesh build output:  ['Submitted batch job 4814475']
4814475


## Check on mesh build status
Run the code box below to get the current status of the mesh build job. You can run it multiple times for the most recent status.

In [101]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
--------------------------------
JobId=4814475 JobName=projectname
   UserId=myashar(43974) GroupId=ucb(501) MCS_label=N/A
   Priority=387228 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=COMPLETED Reason=None Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:01:14 TimeLimit=00:30:00 TimeMin=N/A
   SubmitTime=2019-07-20T16:31:51 EligibleTime=2019-07-20T16:31:51
   StartTime=2019-07-20T16:32:38 EndTime=2019-07-20T16:33:52 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   LastSchedEval=2019-07-20T16:32:38
   Partition=savio AllocNode:Sid=jupyter:425871
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0070.savio1
   BatchHost=n0070.savio1
   NumNodes=1 NumCPUs=20 NumTasks=0 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1,billing=20
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPUsNode=1 MinMemoryNode=62

## Display log message

As an optional step, you can run the code below to see the log file once the job is done.

In [102]:
# print the log file if necessary
slurmlogfilename = 'slurm-' + jobId + '.out'
with open(slurmlogfilename, 'r') as fin:
    print(fin.read(), end="")

Enabling debugging
Ending argument loop
Singularity version: 2.6-dist
Exec'ing: /usr/libexec/singularity/cli/exec.exec
Evaluating args: '-B /global/scratch/myashar/projectname/:/scratch/ /global/scratch/myashar/container/photoscan_1_4_3.simg /scratch/execscript.sh -platform offscreen'
VERBOSE [U=43974,P=76523]  message_init()                            Set messagelevel to: 5
VERBOSE [U=43974,P=76523]  singularity_config_parse()                Initialize configuration file: /etc/singularity/singularity.conf
DEBUG   [U=43974,P=76523]  singularity_config_parse()                Starting parse of configuration file /etc/singularity/singularity.conf
VERBOSE [U=43974,P=76523]  singularity_config_parse()                Got config key allow setuid = 'yes'
VERBOSE [U=43974,P=76523]  singularity_config_parse()                Got config key max loop devices = '256'
VERBOSE [U=43974,P=76523]  singularity_config_parse()                Got config key allow pid ns = 'yes'
VERBOSE [U=43974,P=76523]  si

## Update project on Box
The next code box will return the project file to Box so you can make additional tweaks and changes offline.

In [104]:
update_project_file_in_box()

find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
Box folder id:  53223729428
project file list:  [<Box File - 317022687393 (projectname.psx)>]
datazip file list:  [<Box File - 317017752277 (projectname.files.zip)>, <Box File - 319186364581 (README.docx)>, <Box File - 319043117371 (slurm-generate_model_output.txt)>, <Box File - 319044387168 (slurm-build_texture.txt)>, <Box File - 319044323866 (slurm-export_photomosaic_format_output.txt)>, <Box File - 319035518579 (slurm-export_dense_cloud.txt)>, <Box File - 319043284628 (slurm-pull_images_from_Box.txt)>, <Box File - 319043305927 (slurm-build_dense_cloud.txt)>, <Box File - 319041408570 (slurm-build_mesh.txt)>]
project file id:  317022687393
data zip file list:  317017752277
begin project file update.
update psx result:  <Box File - 317022687393>    update zip result:  <Box File - 317017752277>


## OFFLINE: Mesh cleanup
If you need to add marker points in the Photoscan file (.psx), download the latest project file from Box, make the changes, and upload the edited .psx file back to Box.

# Build Texture

First, retrieve the project file from Box in case it has been updated.

In [105]:
retrieve_project_file()

find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
downloading:  projectname.files.zip
data zip file download complete.
downloading:  projectname.psx
new file:  <_io.BufferedWriter name='/global/scratch/myashar/projectname/projectname.psx'>
project file download complete.


## Choose parameters
Choose the data source, and specify a coordinate system if desired, after running the next code box.

In [106]:
gmchoice=widgets.Dropdown(description='UV Mapping Mode', value='GenericMapping', options= ['GenericMapping', 'OrthophotoMapping', 'AdaptiveOrthophotoMapping', 'SphericalMapping',
'CameraMapping'])
ccchoice=widgets.Dropdown(description='Color Correction', value='True', options= ['True', 'False'] )
ds2choice=widgets.Dropdown(description='Data source', value='ModelData', options=['PointCloudData', 'DenseCloudData', 'DepthMapsData', 'ModelData', 'TiledModelData', 'ElevationData',
'OrthomosaicData']  )

In [107]:
interact(chc, txt=gmchoice)
interact(chc, txt=ds2choice)
interact(chc, txt=ccchoice)
interact(chc, txt=ds2choice)

interactive(children=(Dropdown(description='UV Mapping Mode', options=('GenericMapping', 'OrthophotoMapping', …

interactive(children=(Dropdown(description='Data source', index=3, options=('PointCloudData', 'DenseCloudData'…

interactive(children=(Dropdown(description='Color Correction', options=('True', 'False'), value='True'), Outpu…

interactive(children=(Dropdown(description='Data source', index=3, options=('PointCloudData', 'DenseCloudData'…

<function __main__.chc(txt)>

In [108]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildUV(mapping=PhotoScan.{}) \n\
chunk.calibrateColors( source_data=PhotoScan.DataSource.{}, color_balance={} ) \n\
chunk.buildTexture() \n\
doc.save(\"{}\") \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildOrthomosaic( surface=PhotoScan.DataSource.{} ) \n\
doc.save(\"{}\") \n\
print( "stop time: ", time.time()) \n'

output = template.format(singularitymountfolder + projectFile, gmchoice.value, ds2choice.value, ccchoice.value, singularitymountfolder + projectFile, singularitymountfolder + projectFile, ds2choice.value, singularitymountfolder + projectFile)

print('output:', output)

with open(commandScript, 'w') as f:  
    f.write(output)


output: #!/usr/bin/env python3 
import PhotoScan 
import time 
print( "start time: ", time.time()) 
doc = PhotoScan.app.document 
doc.open("/scratch//projectname.psx") 
chunk = doc.chunk 
chunk.buildUV(mapping=PhotoScan.GenericMapping) 
chunk.calibrateColors( source_data=PhotoScan.DataSource.ModelData, color_balance=True ) 
chunk.buildTexture() 
doc.save("/scratch//projectname.psx") 
doc.open("/scratch//projectname.psx") 
chunk = doc.chunk 
chunk.buildOrthomosaic( surface=PhotoScan.DataSource.ModelData ) 
doc.save("/scratch//projectname.psx") 
print( "stop time: ", time.time()) 



## Run texture build

In [109]:
os.system('export XAUTHORITY=/global/home/users/myashar/.Xauthority')
os.system('export DISPLAY=:0')
os.environ['QT_QPA_PLATFORM']='offscreen'
os.system("module load qt")
os.chdir(runFolder)

out = !sbatch slurmscript.sh   
    
print ('  texture and orthomosaic build output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)    

  texture and orthomosaic build output:  ['Submitted batch job 4814495']
4814495


In [111]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4814495     savio projectn  myashar PD       0:00      1 (Priority)
--------------------------------
JobId=4814495 JobName=projectname
   UserId=myashar(43974) GroupId=ucb(501) MCS_label=N/A
   Priority=387240 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=PENDING Reason=Priority Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:00:00 TimeLimit=00:30:00 TimeMin=N/A
   SubmitTime=2019-07-20T17:29:35 EligibleTime=2019-07-20T17:29:35
   StartTime=2019-07-20T17:33:43 EndTime=2019-07-20T18:03:43 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   LastSchedEval=2019-07-20T17:33:30
   Partition=savio AllocNode:Sid=jupyter:425871
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=(null) SchedNodeList=n0067.savio1
   NumNodes=1 NumCPUs=1 NumTasks=1 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=1,mem=62.50G,node=1
   Socks/Node=* Ntasks

## Display log message

As an optional step, you can run the code below to see the log file once the job is done.

In [112]:
# print the log file if necessary
slurmlogfilename = 'slurm-' + jobId + '.out'
with open(slurmlogfilename, 'r') as fin:
    print(fin.read(), end="")

Enabling debugging
Ending argument loop
Singularity version: 2.6-dist
Exec'ing: /usr/libexec/singularity/cli/exec.exec
Evaluating args: '-B /global/scratch/myashar/projectname/:/scratch/ /global/scratch/myashar/container/photoscan_1_4_3.simg /scratch/execscript.sh -platform offscreen'
VERBOSE [U=43974,P=104007] message_init()                            Set messagelevel to: 5
VERBOSE [U=43974,P=104007] singularity_config_parse()                Initialize configuration file: /etc/singularity/singularity.conf
DEBUG   [U=43974,P=104007] singularity_config_parse()                Starting parse of configuration file /etc/singularity/singularity.conf
VERBOSE [U=43974,P=104007] singularity_config_parse()                Got config key allow setuid = 'yes'
VERBOSE [U=43974,P=104007] singularity_config_parse()                Got config key max loop devices = '256'
VERBOSE [U=43974,P=104007] singularity_config_parse()                Got config key allow pid ns = 'yes'
VERBOSE [U=43974,P=104007] si

## Update project on Box

In [114]:
update_project_file_in_box()

find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
Box folder id:  53223729428
project file list:  [<Box File - 317022687393 (projectname.psx)>]
datazip file list:  [<Box File - 317017752277 (projectname.files.zip)>, <Box File - 319186364581 (README.docx)>, <Box File - 319043117371 (slurm-generate_model_output.txt)>, <Box File - 319044387168 (slurm-build_texture.txt)>, <Box File - 319044323866 (slurm-export_photomosaic_format_output.txt)>, <Box File - 319035518579 (slurm-export_dense_cloud.txt)>, <Box File - 319043284628 (slurm-pull_images_from_Box.txt)>, <Box File - 319043305927 (slurm-build_dense_cloud.txt)>, <Box File - 319041408570 (slurm-build_mesh.txt)>]
project file id:  317022687393
data zip file list:  317017752277
begin project file update.
update psx result:  <Box File - 317022687393>    update zip result:  <Box File - 317017752277>


# Export results

## Export the orthomosaic

## Choose parameters
Choose the image format after running the next code box.

In [115]:
# QUINN guessing at some of these extensions
ifextension = {'ImageFormatJPEG':'.jpg', 'ImageFormatTIFF':'.tiff', 'ImageFormatPNG':'.png', 'ImageFormatBMP':'.bmp', 'ImageFormatEXR':'.exr',
'ImageFormatPNM':'.pnm', 'ImageFormatSGI':'.sgi', 'ImageFormatCR2':'.cr2', 'ImageFormatSEQ':'.seq', 'ImageFormatARA':'.ara'}

ifchoice=widgets.Dropdown(description='Image Format', value='ImageFormatTIFF', options=['ImageFormatJPEG', 'ImageFormatTIFF', 'ImageFormatPNG', 'ImageFormatBMP', 'ImageFormatEXR',
'ImageFormatPNM', 'ImageFormatSGI', 'ImageFormatCR2', 'ImageFormatSEQ', 'ImageFormatARA'] )

In [116]:
interact(chc, txt=ifchoice)

interactive(children=(Dropdown(description='Image Format', index=1, options=('ImageFormatJPEG', 'ImageFormatTI…

<function __main__.chc(txt)>

In [117]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportOrthomosaic(path=\"{}\", image_format=PhotoScan.ImageFormat.{},)  \n' 

orthoFile = projectname + ifextension[ifchoice.value]
output = template.format(singularitymountfolder + projectFile,  singularitymountfolder + orthoFile, ifchoice.value)

print('output:', output)

with open(commandScript, 'w') as f:
    f.write(output)


output: #!/usr/bin/env python3 
import PhotoScan 
doc = PhotoScan.app.document 
doc.open("/scratch//projectname.psx") 
chunk = doc.chunk 
chunk.exportOrthomosaic(path="/scratch/projectname.tiff", image_format=PhotoScan.ImageFormat.ImageFormatTIFF,)  



## Submit job to export the orthomosaic format output

In [118]:
out = !sbatch slurmscript.sh   
        
print (' Export the orthomosaic format output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)  

 Export the orthomosaic format output:  ['Submitted batch job 4814740']
4814740


In [119]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
--------------------------------
JobId=4814740 JobName=projectname
   UserId=myashar(43974) GroupId=ucb(501) MCS_label=N/A
   Priority=1001000 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=COMPLETED Reason=None Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:00:06 TimeLimit=00:30:00 TimeMin=N/A
   SubmitTime=2019-07-21T00:23:20 EligibleTime=2019-07-21T00:23:20
   StartTime=2019-07-21T00:23:20 EndTime=2019-07-21T00:23:26 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   LastSchedEval=2019-07-21T00:23:20
   Partition=savio AllocNode:Sid=jupyter:425871
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0015.savio1
   BatchHost=n0015.savio1
   NumNodes=1 NumCPUs=20 NumTasks=0 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1,billing=20
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPUsNode=1 MinMemoryNode=6

## Display log message

As an optional step, you can run the code below to see the log file once the job is done.

In [124]:
# print the log file if necessary
slurmlogfilename = 'slurm-' + jobId + '.out'
with open(slurmlogfilename, 'r') as fin:
    print(fin.read(), end="")

Enabling debugging
Ending argument loop
Singularity version: 2.6-dist
Exec'ing: /usr/libexec/singularity/cli/exec.exec
Evaluating args: '-B /global/scratch/myashar/projectname/:/scratch/ /global/scratch/myashar/container/photoscan_1_4_3.simg /scratch/execscript.sh -platform offscreen'
VERBOSE [U=43974,P=72899]  message_init()                            Set messagelevel to: 5
VERBOSE [U=43974,P=72899]  singularity_config_parse()                Initialize configuration file: /etc/singularity/singularity.conf
DEBUG   [U=43974,P=72899]  singularity_config_parse()                Starting parse of configuration file /etc/singularity/singularity.conf
VERBOSE [U=43974,P=72899]  singularity_config_parse()                Got config key allow setuid = 'yes'
VERBOSE [U=43974,P=72899]  singularity_config_parse()                Got config key max loop devices = '256'
VERBOSE [U=43974,P=72899]  singularity_config_parse()                Got config key allow pid ns = 'yes'
VERBOSE [U=43974,P=72899]  si

## Move the orthomosaic file to Box

In [125]:
newFolderId = find_folder_id(boxProjectFolder)
upload_folder = client.folder(folder_id=newFolderId).get()
orthoUploaded = upload_folder.upload( runFolder + orthoFile)  
print ("orthomosaic file: ", orthoFile)
print ("orthomosaic file id: ", orthoUploaded['id'] )

find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
orthomosaic file:  projectname.tiff
orthomosaic file id:  494460543876


## Generate the Model export


## Choose parameters
Choose the output file extensions and model formats after running the next code box.

In [126]:

# QUINN guessing at some of these extensions
mfextension = { 'ModelFormatOBJ':'.obj', 'ModelFormat3DS':'.3ds', 'ModelFormatVRML':'.vrml', 'ModelFormatPLY':'.ply', 'ModelFormatCOLLADA':'.col',
'ModelFormatU3D':'.u3d', 'ModelFormatPDF':'.pdf', 'ModelFormatDXF':'.dxf', 'ModelFormatFBX':'.fbx', 'ModelFormatKMZ':'.kmz',
'ModelFormatCTM':'.ctm', 'ModelFormatSTL':'.stl', 'ModelFormatDXF_3DF':'.3df', 'ModelFormatTLS':'.tls' }

mfchoice=widgets.Dropdown(description='Model Format', value='ModelFormatOBJ', options= ['ModelFormatOBJ', 'ModelFormat3DS', 'ModelFormatVRML', 'ModelFormatPLY', 'ModelFormatCOLLADA',
'ModelFormatU3D', 'ModelFormatPDF', 'ModelFormatDXF', 'ModelFormatFBX', 'ModelFormatKMZ',
'ModelFormatCTM', 'ModelFormatSTL', 'ModelFormatDXF_3DF', 'ModelFormatTLS'] )

In [127]:
interact(chc, txt=mfchoice)

interactive(children=(Dropdown(description='Model Format', options=('ModelFormatOBJ', 'ModelFormat3DS', 'Model…

<function __main__.chc(txt)>

In [128]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportModel(path=\"{}\", format=PhotoScan.ModelFormat.{})  \n' 

modelFile = projectname + mfextension[mfchoice.value]
output = template.format(singularitymountfolder + projectFile, singularitymountfolder + modelFile, mfchoice.value)
print('output:', output)

with open(commandScript, 'w') as f:
    f.write(output)


output: #!/usr/bin/env python3 
import PhotoScan 
doc = PhotoScan.app.document 
doc.open("/scratch//projectname.psx") 
chunk = doc.chunk 
chunk.exportModel(path="/scratch/projectname.obj", format=PhotoScan.ModelFormat.ModelFormatOBJ)  



## Submit job to generate model output

In [129]:
out = !sbatch slurmscript.sh   
        
print (' Generate model output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)         


 Generate model output:  ['Submitted batch job 4816678']
4816678


In [130]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
--------------------------------
JobId=4816678 JobName=projectname
   UserId=myashar(43974) GroupId=ucb(501) MCS_label=N/A
   Priority=304763 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=COMPLETED Reason=None Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:00:06 TimeLimit=00:30:00 TimeMin=N/A
   SubmitTime=2019-07-21T16:49:37 EligibleTime=2019-07-21T16:49:37
   StartTime=2019-07-21T16:49:38 EndTime=2019-07-21T16:49:44 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   LastSchedEval=2019-07-21T16:49:38
   Partition=savio AllocNode:Sid=jupyter:425871
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0127.savio1
   BatchHost=n0127.savio1
   NumNodes=1 NumCPUs=20 NumTasks=0 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1,billing=20
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPUsNode=1 MinMemoryNode=62

## Display log message

As an optional step, you can run the code below to see the log file once the job is done.

In [131]:
# print the log file if necessary
slurmlogfilename = 'slurm-' + jobId + '.out'
with open(slurmlogfilename, 'r') as fin:
    print(fin.read(), end="")

Enabling debugging
Ending argument loop
Singularity version: 2.6-dist
Exec'ing: /usr/libexec/singularity/cli/exec.exec
Evaluating args: '-B /global/scratch/myashar/projectname/:/scratch/ /global/scratch/myashar/container/photoscan_1_4_3.simg /scratch/execscript.sh -platform offscreen'
VERBOSE [U=43974,P=29120]  message_init()                            Set messagelevel to: 5
VERBOSE [U=43974,P=29120]  singularity_config_parse()                Initialize configuration file: /etc/singularity/singularity.conf
DEBUG   [U=43974,P=29120]  singularity_config_parse()                Starting parse of configuration file /etc/singularity/singularity.conf
VERBOSE [U=43974,P=29120]  singularity_config_parse()                Got config key allow setuid = 'yes'
VERBOSE [U=43974,P=29120]  singularity_config_parse()                Got config key max loop devices = '256'
VERBOSE [U=43974,P=29120]  singularity_config_parse()                Got config key allow pid ns = 'yes'
VERBOSE [U=43974,P=29120]  si

## Move the OBJ and associated JPG to Box
You can download the files from Box and preview using http://3dviewer.net/

In [134]:
newFolderId = find_folder_id(boxProjectFolder)
upload_folder = client.folder(folder_id=newFolderId).get()
objUploaded = upload_folder.upload(runFolder + modelFile)  
print ("obj file id: ", objUploaded['id'] )
jpgUploaded = upload_folder.upload(runFolder + jpgFile)  
print ("jpeg file id: ", jpgUploaded['id'] )
print("obj file: ", modelFile)
print("jpeg file: ", jpgFile)

find_folder_id folder_name:  myBoxFolderName
find_folder_id folderlist:  [<Box Folder - 53223729428 (myBoxFolderName)>, <Box Folder - 52394687398 (myBoxFolderName_Test1)>]
obj file id:  494632908572
jpeg file id:  494626805638
obj file:  projectname.obj
jpeg file:  projectname.jpg


# Software license
This software is available under the terms of the Educational Community License, Version 2.0 (ECL 2.0). This software is Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley").

The text of the ECL license is reproduced below.

Educational Community License, Version 2.0
*************************************
Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley")

Educational Community License, Version 2.0, April 2007

The Educational Community License version 2.0 ("ECL") consists of the
Apache 2.0 license, modified to change the scope of the patent grant in
section 3 to be specific to the needs of the education communities using
this license. The original Apache 2.0 license can be found at:[http://www.apache.org/licenses/LICENSE-2.0]